# Manual Calculation Workbook: Frequent Pattern Analysis from Scratch

This notebook implements frequent pattern analysis using pure Python without external libraries (except basic collections). Follow along to understand the core algorithms.

## Dataset Setup

We'll use the same small dataset from our manual calculations to verify our implementation.

In [ ]:
# Sample transaction database
transactions = [
    ['Milk', 'Bread', 'Eggs'],           # T1
    ['Bread', 'Eggs', 'Butter'],        # T2
    ['Milk', 'Eggs'],                   # T3
    ['Milk', 'Bread', 'Eggs', 'Butter'], # T4
    ['Milk', 'Bread'],                  # T5
    ['Bread', 'Eggs']                   # T6
]

print("Transaction Database:")
for i, transaction in enumerate(transactions, 1):
    print(f"T{i}: {transaction}")
    
print(f"\nTotal transactions: {len(transactions)}")

## Step 1: Calculate Support for Individual Items

In [ ]:
def calculate_support(itemset, transactions):
    """
    Calculate support for an itemset.
    
    Args:
        itemset: A set or list of items
        transactions: List of transaction lists
    
    Returns:
        Support value (float between 0 and 1)
    """
    if isinstance(itemset, list):
        itemset = set(itemset)
    
    count = 0
    for transaction in transactions:
        if itemset.issubset(set(transaction)):
            count += 1
    
    return count / len(transactions)

# Get all unique items
all_items = set()
for transaction in transactions:
    all_items.update(transaction)

print("Individual Item Support:")
print("-" * 30)
item_support = {}
for item in sorted(all_items):
    support = calculate_support([item], transactions)
    item_support[item] = support
    print(f"{item:8}: {support:.3f} ({support*100:.1f}%)")

## Step 2: Implement Apriori Algorithm from Scratch

In [ ]:
from itertools import combinations

def generate_candidates(frequent_itemsets, k):
    """
    Generate candidate k-itemsets from frequent (k-1)-itemsets.
    
    Args:
        frequent_itemsets: List of frequent (k-1)-itemsets
        k: Size of candidates to generate
    
    Returns:
        List of candidate k-itemsets
    """
    candidates = []
    
    # Convert to sorted tuples for consistent comparison
    itemsets = [tuple(sorted(itemset)) for itemset in frequent_itemsets]
    
    # Generate candidates by joining itemsets
    for i in range(len(itemsets)):
        for j in range(i + 1, len(itemsets)):
            # Join if first k-2 items are the same
            if itemsets[i][:-1] == itemsets[j][:-1]:
                candidate = tuple(sorted(set(itemsets[i]) | set(itemsets[j])))
                if len(candidate) == k:
                    candidates.append(candidate)
    
    return list(set(candidates))  # Remove duplicates

def apriori_algorithm(transactions, min_support):
    """
    Implement the Apriori algorithm from scratch.
    
    Args:
        transactions: List of transaction lists
        min_support: Minimum support threshold
    
    Returns:
        Dictionary with frequent itemsets by level
    """
    # Get all unique items
    all_items = set()
    for transaction in transactions:
        all_items.update(transaction)
    
    frequent_itemsets = {}
    k = 1
    
    # Level 1: Individual items
    candidates = [[item] for item in all_items]
    
    while candidates:
        print(f"\n--- Level {k} ---")
        print(f"Candidates: {len(candidates)}")
        
        # Calculate support for candidates
        frequent_k = []
        for candidate in candidates:
            support = calculate_support(candidate, transactions)
            print(f"  {candidate}: support = {support:.3f}")
            
            if support >= min_support:
                frequent_k.append(candidate)
        
        if frequent_k:
            frequent_itemsets[k] = frequent_k
            print(f"Frequent {k}-itemsets: {len(frequent_k)}")
            for itemset in frequent_k:
                print(f"  {itemset}")
            
            # Generate candidates for next level
            k += 1
            candidates = generate_candidates(frequent_k, k)
        else:
            print(f"No frequent {k}-itemsets found. Stopping.")
            break
    
    return frequent_itemsets

# Run Apriori algorithm
min_support = 0.5  # 50%
print(f"Running Apriori Algorithm with minimum support = {min_support}")
print("=" * 50)

frequent_itemsets = apriori_algorithm(transactions, min_support)

## Step 3: Generate Association Rules

In [ ]:
def generate_association_rules(frequent_itemsets, transactions, min_confidence=0.5):
    """
    Generate association rules from frequent itemsets.
    
    Args:
        frequent_itemsets: Dictionary of frequent itemsets by level
        transactions: List of transaction lists
        min_confidence: Minimum confidence threshold
    
    Returns:
        List of association rules with metrics
    """
    rules = []
    
    # Generate rules from itemsets of size 2 or more
    for level in range(2, len(frequent_itemsets) + 1):
        if level not in frequent_itemsets:
            continue
            
        for itemset in frequent_itemsets[level]:
            # Generate all possible antecedent-consequent pairs
            items = list(itemset)
            
            # For each possible split of the itemset
            for i in range(1, len(items)):
                for antecedent in combinations(items, i):
                    consequent = tuple(item for item in items if item not in antecedent)
                    
                    # Calculate metrics
                    support_full = calculate_support(itemset, transactions)
                    support_antecedent = calculate_support(antecedent, transactions)
                    support_consequent = calculate_support(consequent, transactions)
                    
                    confidence = support_full / support_antecedent
                    lift = confidence / support_consequent
                    
                    if confidence >= min_confidence:
                        rules.append({
                            'antecedent': antecedent,
                            'consequent': consequent,
                            'support': support_full,
                            'confidence': confidence,
                            'lift': lift
                        })
    
    return rules

# Generate association rules
min_confidence = 0.5  # 50%
rules = generate_association_rules(frequent_itemsets, transactions, min_confidence)

print(f"\nAssociation Rules (min_confidence = {min_confidence}):")
print("=" * 70)
print(f"{'Rule':<25} {'Support':<10} {'Confidence':<12} {'Lift':<10}")
print("-" * 70)

for rule in rules:
    antecedent = ', '.join(rule['antecedent'])
    consequent = ', '.join(rule['consequent'])
    rule_str = f"{antecedent} → {consequent}"
    
    print(f"{rule_str:<25} {rule['support']:<10.3f} {rule['confidence']:<12.3f} {rule['lift']:<10.3f}")

## Step 4: Detailed Manual Verification

In [ ]:
def verify_manual_calculations():
    """
    Verify our calculations match the manual work from the README.
    """
    print("Manual Verification:")
    print("=" * 40)
    
    # Verify individual item support
    print("\n1. Individual Item Support:")
    expected = {'Milk': 4/6, 'Bread': 5/6, 'Eggs': 5/6, 'Butter': 2/6}
    
    for item, exp_support in expected.items():
        calc_support = calculate_support([item], transactions)
        match = "✓" if abs(calc_support - exp_support) < 0.001 else "✗"
        print(f"  {item}: Expected {exp_support:.3f}, Got {calc_support:.3f} {match}")
    
    # Verify 2-itemset support
    print("\n2. 2-Itemset Support:")
    expected_2 = {
        ('Milk', 'Bread'): 3/6,
        ('Milk', 'Eggs'): 3/6,
        ('Bread', 'Eggs'): 4/6
    }
    
    for itemset, exp_support in expected_2.items():
        calc_support = calculate_support(itemset, transactions)
        match = "✓" if abs(calc_support - exp_support) < 0.001 else "✗"
        print(f"  {itemset}: Expected {exp_support:.3f}, Got {calc_support:.3f} {match}")
    
    # Verify specific rule calculations
    print("\n3. Association Rule Verification:")
    
    # Bread → Eggs rule
    support_bread_eggs = calculate_support(['Bread', 'Eggs'], transactions)
    support_bread = calculate_support(['Bread'], transactions)
    support_eggs = calculate_support(['Eggs'], transactions)
    
    confidence = support_bread_eggs / support_bread
    lift = confidence / support_eggs
    
    print(f"  Bread → Eggs:")
    print(f"    Support: {support_bread_eggs:.3f}")
    print(f"    Confidence: {confidence:.3f}")
    print(f"    Lift: {lift:.3f}")
    print(f"    Expected: Support=0.667, Confidence=0.800, Lift=0.960")

verify_manual_calculations()

## Step 5: Interactive Experimentation

In [ ]:
def experiment_with_thresholds():
    """
    Experiment with different support and confidence thresholds.
    """
    print("Threshold Experimentation:")
    print("=" * 40)
    
    support_thresholds = [0.3, 0.4, 0.5, 0.6]
    
    for min_sup in support_thresholds:
        print(f"\nMinimum Support = {min_sup}:")
        freq_itemsets = apriori_algorithm(transactions, min_sup)
        
        # Count total frequent itemsets
        total_itemsets = sum(len(itemsets) for itemsets in freq_itemsets.values())
        print(f"  Total frequent itemsets: {total_itemsets}")
        
        # Generate rules
        rules = generate_association_rules(freq_itemsets, transactions, min_confidence=0.5)
        print(f"  Total rules (confidence ≥ 0.5): {len(rules)}")

# Uncomment to run the experiment
# experiment_with_thresholds()

## Step 6: Summary and Insights

In [ ]:
def summarize_analysis():
    """
    Provide a summary of the analysis and key insights.
    """
    print("Analysis Summary:")
    print("=" * 40)
    
    print("\nKey Findings:")
    print("1. With 50% minimum support:")
    print("   - Butter was eliminated (only 33.3% support)")
    print("   - 3 frequent 1-itemsets: Milk, Bread, Eggs")
    print("   - 3 frequent 2-itemsets: {Milk,Bread}, {Milk,Eggs}, {Bread,Eggs}")
    print("   - 0 frequent 3-itemsets")
    
    print("\n2. Association Rules:")
    print("   - All lift values < 1.0 (weak or negative associations)")
    print("   - Strongest rule: Bread → Eggs (80% confidence)")
    print("   - Most frequent rule: Bread ↔ Eggs (66.7% support)")
    
    print("\n3. Business Implications:")
    print("   - Bread and Eggs are commonly bought together")
    print("   - Milk appears in many transactions but weaker associations")
    print("   - Butter is a niche item (low frequency)")
    
    print("\n4. Algorithm Performance:")
    print("   - Manual implementation matches library results")
    print("   - Apriori efficiently pruned search space")
    print("   - No 3-itemsets needed evaluation due to low 2-itemset support")

summarize_analysis()

## Next Steps

Now that you understand the manual process:

1. **Try Different Datasets**: Experiment with your own transaction data
2. **Adjust Thresholds**: See how different support/confidence values affect results
3. **Compare with Libraries**: Use mlxtend to verify your understanding
4. **Scale Up**: Apply to larger datasets using optimized algorithms
5. **Business Application**: Interpret rules in real-world contexts

The manual understanding you've gained here is fundamental to using automated tools effectively!